# fpbase

In [62]:
import pandas as pd
fluor=pd.read_csv("../data/fpbase_raw_20-9-2.csv")
fluor=fluor.dropna(subset=['seq'])
fluor["pdbs"]=fluor[fluor.columns[fluor.columns.str.startswith("pdb")]].apply(lambda x: ','.join(x.dropna()), axis=1)
fluor=fluor[['name','doi','uniprot','genbank','pdbs','seq']]
fluor=fluor.set_index('name')
fluor.index=fluor.index.str.replace(" ","_")
fluor.index=fluor.index.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
fluor['description']=fluor[['doi','uniprot','genbank','pdbs']].apply(lambda x: ' | '.join(x.dropna()), axis=1)
fluor

,doi,uniprot,genbank,pdbs,seq,description
name,,,,,,
10B,10.1126/science.273.5280.1392,NaN,NaN,,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,10.1126/science.273.5280.1392 |
11,10.1126/science.273.5280.1392,NaN,NaN,,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,10.1126/science.273.5280.1392 |
22G,10.1126/science.1102506,NaN,NaN,2Z6X,MSVIKPDMKIKLRMEGAVNGHPFAIEGVGLGKPFEGKQSMDLKVKE...,10.1126/science.1102506 | 2Z6X
(3-F)Tyr-EGFP,10.1002/cbic.200300818,NaN,NaN,1RRX,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,10.1002/cbic.200300818 | 1RRX
5B,10.1126/science.273.5280.1392,NaN,NaN,,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,10.1126/science.273.5280.1392 |
...,...,...,...,...,...,...
zGFP,10.1007/s12033-008-9131-y,B2ZAG1,ACD03133,,MAYSKQGLTDNMTMKYQMEGCVDGHQFVITGHGKGNPFTGKQTLNL...,10.1007/s12033-008-9131-y | B2ZAG1 | ACD03133 |
zoan2RFP,10.1073/pnas.062552299,Q8T4U4,AAL23574,,MAHSKHGLTDDMTMHFRMEGCVDGHKFVIEGNGNGNPFKGKQFINL...,10.1073/pnas.062552299 | Q8T4U4 | AAL23574 |
zRFP,10.1007/s12033-008-9131-y,B2ZAG2,ACD03134,,MAHSKHGLTNDMTMKFRMEGCVDGHQFVITGHGNGNPFTGEQTINL...,10.1007/s12033-008-9131-y | B2ZAG2 | ACD03134 |


In [64]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
seqs=[]
for name in fluor.index:
        seq=fluor.loc[name]['seq']
        desc=""#fluor.loc[name]['description']
        seqs.append(SeqRecord(Seq(seq),id=name,description=desc))
SeqIO.write(seqs,'../data/fpbase_20-9-2.fasta', 'fasta')

721

In [65]:
import subprocess
from tempfile import NamedTemporaryFile
def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
bash('makeblastdb -in ../data/fpbase_20-9-2.fasta -out ../BLAST_dbs/fpbase_20-9-2 -dbtype prot -parse_seqids')

'\n\nBuilding a new DB, current time: 09/14/2020 22:19:59\nNew DB name:   /Users/mattmcguffie/Documents/GitHub/pLannotate/BLAST_dbs/fpbase_20-9-2\nNew DB title:  ../data/fpbase_20-9-2.fasta\nSequence type: Protein\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 721 sequences in 0.042753 seconds.\n'

# snapgene features

In [220]:
from Bio import SeqIO
import glob
gbks=[]
for infile_loc in glob.glob('../data/addgene_features_20-9-3/*.gb'):
    with open(infile_loc,'r') as file_handle:
        record_dict = SeqIO.to_dict(SeqIO.parse(file_handle, 'gb'))
    name=infile_loc.split("/")[-1].split(".")[0]
    gbk=record_dict[list(record_dict.keys())[0]]
    #gbk.description=gbk.features[1].description
    gbk.name=gbk.features[1].qualifiers['label'][0]
    gbk.id=name
    gbks.append(gbk)

In [56]:
featureTable=[]
import glob
for infile_loc in glob.glob('../data/addgene_features_20-9-3/*.gb'):
    with open(infile_loc,'r') as file_handle:
        record_dict = SeqIO.to_dict(SeqIO.parse(file_handle, 'gb'))
    gbk=record_dict[list(record_dict.keys())[0]]
    
    name=infile_loc.split("/")[-1].split(".gb")[0]
    curr={"name":name,"type":gbk.features[1].type}
    keys=list(gbk.features[1].qualifiers.keys())
    for key in keys:
        curr[key]=gbk.features[1].qualifiers[key][0]
    curr['seq']=str(gbk.seq)
    
    featureTable.append(curr)

featureTable=pd.DataFrame(featureTable)
featureTable['canon_name']=featureTable['name'].str.replace(" ","_").str.replace(",","_")
featureTable

,name,type,label,note,seq,codon_start,product,translation,bound_moiety,gene,...,mobile_element_type,regulatory_class,direction,allele,number,gene_synonym,db_xref,protein_id,ncRNA_class,canon_name
0,rrnG terminator,terminator,rrnG terminator,transcription terminator from the E. coli ribo...,GCATTGGCGCAGAAAAAAATGCCTGATGCGACGCTGCGCGTCTTAT...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rrnG_terminator
1,myr,CDS,myr,NaN,ATGGGGAGTAGCAAGAGCAAGCCTAAGGACCCCAGCCAGCGC,1,N-myristoylation signal from Src kinase (Pellm...,MGSSKSKPKDPSQR,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,myr
2,CuO,protein_bind,CuO,CymR-binding P1 operator sequence from the p-c...,AGAAACAAACCAACCTGTCTGTATTA,NaN,NaN,NaN,CymR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CuO
3,N15 ori,rep_origin,N15 ori,bidirectional replication origin of coliphage N15,ACGCACGTAAGGTGGAAAAAACCACCATCGAAGTGGATCGCGATTT...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N15_ori
4,Rosa26 left arm,misc_feature,Rosa26 left arm,left homology arm from intron 1 of the mouse R...,CAGTTAACGGCAGCCGGAGTGCGCAGCCGCCGGCAGCCTCGCTCTG...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rosa26_left_arm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,oriT,oriT,NaN,incP origin of transfer,GGGCAGGATAGGTGAAGTAGGCCCACCCGCGAGCGGGTGTTCCTTC...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oriT
1355,Sec61-beta,CDS,Sec61-beta,NaN,ATGCCTGGTCCGACCCCCAGTGGCACTAACGTGGGATCCTCAGGGC...,1,human Sec61-beta subunit of the translocon in ...,MPGPTPSGTNVGSSGRSPSKAVAARAAGSTVRQRKNASCGTRSAGR...,NaN,SEC61B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sec61-beta
1356,BFP,CDS,BFP,mammalian codon-optimized,ATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCC...,1,blue variant of GFP,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BFP
1357,CBRluc,CDS,CBRluc,human codon-optimized,ATGGTAAAGCGTGAGAAAAATGTCATCTATGGCCCTGAGCCTCTCC...,1,red-emitting variant of click beetle luciferase,MVKREKNVIYGPEPLHPLEDLTAGEMLFRALRKHSHLPQALVDVVG...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBRluc


In [57]:
featureTable.to_csv("../data/snapgene_features_20-9-3.csv",sep="\t",index=None)

In [58]:
featureTable=pd.read_csv("../data/snapgene_features_20-9-3.csv",sep="\t")

In [59]:

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
seqs=[]
for index in featureTable[featureTable['type']=='CDS'].index:
        seq=featureTable.loc[index]['seq']
        name=featureTable.loc[index]['canon_name']
        seqs.append(SeqRecord(Seq(seq),id=name,description=""))
SeqIO.write(seqs,'../data/addgene_CDS_20-9-3.fasta', 'fasta')

import subprocess
from tempfile import NamedTemporaryFile
def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
#bash('makeblastdb -blastdb_version 5 -in ../data/addgene_CDS_20-9-3.fasta -out ../BLAST_dbs/addgene_CDS_20-9-3 -dbtype prot -parse_seqids')

### Potentially problematic
if using blastx, different nuc seqs will all hit

In [327]:
for ele in featureTable[featureTable['translation']=='MDKKYSIGLDIGTNSVGWAVITDEYKVPSKKFKVLGNTDRHSIKKNLIGALLFDSGETAEATRLKRTARRRYTRRKNRICYLQEIFSNEMAKVDDSFFHRLEESFLVEEDKKHERHPIFGNIVDEVAYHEKYPTIYHLRKKLVDSTDKADLRLIYLALAHMIKFRGHFLIEGDLNPDNSDVDKLFIQLVQTYNQLFEENPINASGVDAKAILSARLSKSRRLENLIAQLPGEKKNGLFGNLIALSLGLTPNFKSNFDLAEDAKLQLSKDTYDDDLDNLLAQIGDQYADLFLAAKNLSDAILLSDILRVNTEITKAPLSASMIKRYDEHHQDLTLLKALVRQQLPEKYKEIFFDQSKNGYAGYIDGGASQEEFYKFIKPILEKMDGTEELLVKLNREDLLRKQRTFDNGSIPHQIHLGELHAILRRQEDFYPFLKDNREKIEKILTFRIPYYVGPLARGNSRFAWMTRKSEETITPWNFEEVVDKGASAQSFIERMTNFDKNLPNEKVLPKHSLLYEYFTVYNELTKVKYVTEGMRKPAFLSGEQKKAIVDLLFKTNRKVTVKQLKEDYFKKIECFDSVEISGVEDRFNASLGTYHDLLKIIKDKDFLDNEENEDILEDIVLTLTLFEDREMIEERLKTYAHLFDDKVMKQLKRRRYTGWGRLSRKLINGIRDKQSGKTILDFLKSDGFANRNFMQLIHDDSLTFKEDIQKAQVSGQGDSLHEHIANLAGSPAIKKGILQTVKVVDELVKVMGRHKPENIVIEMARENQTTQKGQKNSRERMKRIEEGIKELGSQILKEHPVENTQLQNEKLYLYYLQNGRDMYVDQELDINRLSDYDVDHIVPQSFLKDDSIDNKVLTRSDKNRGKSDNVPSEEVVKKMKNYWRQLLNAKLITQRKFDNLTKAERGGLSELDKAGFIKRQLVETRQITKHVAQILDSRMNTKYDENDKLIREVKVITLKSKLVSDFRKDFQFYKVREINNYHHAHDAYLNAVVGTALIKKYPKLESEFVYGDYKVYDVRKMIAKSEQEIGKATAKYFFYSNIMNFFKTEITLANGEIRKRPLIETNGETGEIVWDKGRDFATVRKVLSMPQVNIVKKTEVQTGGFSKESILPKRNSDKLIARKKDWDPKKYGGFDSPTVAYSVLVVAKVEKGKSKKLKSVKELLGITIMERSSFEKNPIDFLEAKGYKEVKKDLIIKLPKYSLFELENGRKRMLASAGELQKGNELALPSKYVNFLYLASHYEKLKGSPEDNEQKQLFVEQHKHYLDEIIEQISEFSKRVILADANLDKVLSAYNKHRDKPIREQAENIIHLFTLTNLGAPAAFKYFDTTIDRKRYTSTKEVLDATLIHQSITGLYETRIDLSQLGGD']['product']:
    print(ele,end="\n\n")

Cas9 endonuclease from the Streptococcus pyogenes Type II CRISPR/Cas system

Cas9 (Csn1) endonuclease from the Streptococcus pyogenes Type II CRISPR/Cas system

S. pyogenes Cas9, codon optimized for C. elegans with a synthetic intron

Cas9 (Csn1) endonuclease from the Streptococcus pyogenes Type II CRISPR/Cas system

S. pyogenes Cas9, codon optimized for C. elegans with synthetic introns

S. pyogenes Cas9, codon optimized for C. elegans with synthetic introns



In [ ]:
pd.DataFrame()

# Merging BLAST dbs

In [ ]:
blastdb_aliastool -dblist "swissprot addgene_CDS_20-9-3 fpbase_20-9-2" -dbtype prot -title "swissprot + addgene CDSs + fpbase" -out ./merged_CDS